# 1 – Data Prep (Final)

Dieses Notebook erzeugt für eine frei wählbare `EXP_ID` die Labels
und den Trainingsdatensatz für das Zwei-Stufen-XGBoost-Modell.

Es ruft nur die Python-Pipelines auf und enthält bewusst **kein**
Modell-Training oder umfangreiche Visualisierungen.


In [4]:
import sys
from pathlib import Path
import os

cwd = Path.cwd()
project_root = cwd
while not (project_root / 'src').is_dir():
    if project_root.parent == project_root:
        raise RuntimeError("Projektwurzel mit 'src' nicht gefunden.")
    project_root = project_root.parent

print('Erkannte Projektwurzel:', project_root)
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

os.chdir(project_root)
print('Arbeitsverzeichnis gesetzt auf:', Path.cwd())


Erkannte Projektwurzel: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project
Arbeitsverzeichnis gesetzt auf: /Users/jeremynathan/Documents/GitHub/hs2025_ml_project/hs2025_ml_project


In [5]:
# Experiment-Konfiguration – bitte EXP_ID explizit setzen, bevor du das Notebook ausführst.
#
# Dieses Notebook ist aktuell auf das erste hv-Experiment eingestellt,
# das die neue Label-Definition (Treffer irgendwo im Horizont) nutzt.

EXP_ID = 'hv_flex_0_7_result'  # <- hier setzen
assert EXP_ID != 'CHANGE_ME', 'Bitte EXP_ID oben setzen, bevor du weiterläufst.'

# True = mit News-Merge, False = nur Preise
USE_NEWS = True
FEATURE_MODE = 'news+price' if USE_NEWS else 'price_only'

# Wenn True: überschreibt auch die 'latest' Dateien (eurusd_labels.csv, eurusd_*_training.csv)
WRITE_LATEST = False


# Label-Parameter – hv1: 4 Tage, ±0.3 %, nicht strikt,
# max. 1 % gegenläufige Bewegung erlaubt, Treffer reicht,
# sobald die Schwelle innerhalb des Horizonts einmal erreicht wird.
LABEL_PARAMS = dict(
    # Datenquelle: 'mt5' nutzt `data/raw/fx/EURUSD_mt5_D1.csv`
    price_source='yahoo',
    drop_weekends=False,
    horizon_days=15,
    up_threshold=0.02,
    down_threshold=-0.02,
    strict_monotonic=False,
    max_adverse_move_pct=0.004,
    # Neue Option: Wenn True, wird ein Label bereits vergeben,
    # sobald die Schwelle irgendwo innerhalb des Horizonts
    # [t, t+horizon_days] einmal erreicht/ueberschritten wird.
    # Standard=False -> nur Endkurs am Horizonttag entscheidet.
    hit_within_horizon=True,
    # Neu: Wenn True, wird bei hit_within_horizon=True geprüft,
    # ob zuerst die Up- oder Down-Schwelle erreicht wird.
    # Das Label entspricht dann dem *ersten* Treffer (trading-näher).
    first_hit_wins=True,
)


In [6]:
import json
from pathlib import Path

from src.utils.io import DATA_PROCESSED
from src.data.label_eurusd import label_eurusd
from src.data.build_training_set import (
    build_training_dataframe,
    build_price_only_training_dataframe_from_labels,
)

print('EXP_ID:', EXP_ID)
print('FEATURE_MODE:', FEATURE_MODE)
print('WRITE_LATEST:', WRITE_LATEST)
print('LABEL_PARAMS:', LABEL_PARAMS)

# 1) Config-JSON speichern
exp_meta_dir = DATA_PROCESSED / 'experiments'
exp_meta_dir.mkdir(parents=True, exist_ok=True)
exp_config = {
    'exp_id': EXP_ID,
    'feature_mode': FEATURE_MODE,
    'label_params': LABEL_PARAMS,
}
exp_config_path = exp_meta_dir / f'{EXP_ID}_config.json'
with exp_config_path.open('w', encoding='utf-8') as f:
    json.dump(exp_config, f, indent=2)
print('[ok] Experiment-Konfiguration gespeichert unter:', exp_config_path)

# 2) Labels berechnen
labels = label_eurusd(**LABEL_PARAMS)
fx_dir = DATA_PROCESSED / 'fx'
fx_dir.mkdir(parents=True, exist_ok=True)
labels_path_exp = fx_dir / f'eurusd_labels__{EXP_ID}.csv'
labels.to_csv(labels_path_exp)
print('[ok] Labels gespeichert als:', labels_path_exp)
if WRITE_LATEST:
    labels_path_latest = fx_dir / 'eurusd_labels.csv'
    labels.to_csv(labels_path_latest)
    print('[ok] Zusätzlich überschrieben:', labels_path_latest)

# 3) Trainingsdatensatz bauen
if FEATURE_MODE == 'news+price':
    # News-Features sicherstellen (aus Raw-News ableiten, falls CSV fehlt)
    news_path = DATA_PROCESSED / 'news' / 'eodhd_daily_features.csv'
    if not news_path.is_file():
        from src.data.prepare_eodhd_news import build_daily_features, save_daily_features
        df_news = build_daily_features()
        save_daily_features(df_news, news_path)
        print('[ok] News-Tagesfeatures erzeugt:', news_path)

    merged = build_training_dataframe(exp_id=EXP_ID)
    ds_name = f'eurusd_news_training__{EXP_ID}.csv'
    latest_name = 'eurusd_news_training.csv'
else:
    merged = build_price_only_training_dataframe_from_labels(exp_id=EXP_ID)
    ds_name = f'eurusd_price_training__{EXP_ID}.csv'
    latest_name = 'eurusd_price_training.csv'

ds_dir = DATA_PROCESSED / 'datasets'
ds_dir.mkdir(parents=True, exist_ok=True)
train_path_exp = ds_dir / ds_name
merged.to_csv(train_path_exp, index=False)
print('[ok] Trainingsdatensatz gespeichert als:', train_path_exp)
if WRITE_LATEST:
    train_path_latest = ds_dir / latest_name
    merged.to_csv(train_path_latest, index=False)
    print('[ok] Zusätzlich überschrieben:', train_path_latest)


EXP_ID: hv_flex_0_7_result
LABEL_PARAMS: {'price_source': 'yahoo', 'drop_weekends': False, 'horizon_days': 15, 'up_threshold': 0.02, 'down_threshold': -0.02, 'strict_monotonic': False, 'max_adverse_move_pct': 0.004, 'hit_within_horizon': True, 'first_hit_wins': True}
[ok] Experiment-Konfiguration gespeichert unter: data/processed/experiments/hv_flex_0_7_result_config.json
[ok] Labels gespeichert als:
    data/processed/fx/eurusd_labels__hv_flex_0_7_result.csv
    data/processed/fx/eurusd_labels.csv
[ok] Trainingsdatensatz gespeichert als:
    data/processed/datasets/eurusd_news_training__hv_flex_0_7_result.csv
    data/processed/datasets/eurusd_news_training.csv
